# README.

This notebook is the entrypoint for Azure ML enabled training.
In its essence, it connects to Azure ML, makes sure that everything is ready over there, and starts the training.
To that end, this notebook gathers all necessary sourcecodes in a temp-folder, which will be pushed to Azure ML for training.

# Imports.

In [1]:
%reload_ext autoreload
%autoreload 2

import glob2 as glob
from azureml.core import Workspace, Dataset
from azureml.core import Experiment, Environment, ScriptRunConfig
from pathlib import Path
import os
import shutil

from src.constants import REPO_DIR

# Setting screws

In [2]:
dataset_name = "anon-depthmap-95k"  # "anon-depthmap-mini"
experiment_name = "q1-depthmap-plaincnn-height-95k-evidential"
tags = {'USE_WANDB': 'True', 'LR': "0.0000007", 'EDL_COEFF': '1e-2', 'optimizer': 'Adam', 'normalize': 'True: targets minus minimum'}

# Create temp folder and copy code.

Here you have to be very precise, which code to copy.
And most importantly, which code NOT to copy.

In [3]:
code_dir = "src"
paths = glob.glob(os.path.join(code_dir, "*.py"))
paths

['src/config.py',
 'src/constants.py',
 'src/__init__.py',
 'src/model.py',
 'src/train.py',
 'src/config_weight.py']

In [4]:
print("Creating temp folder...")
temp_path = "tmp_train"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

for p in paths:
    shutil.copy(p, temp_path)
print("Done.")

Creating temp folder...
Done.


In [5]:
utils_dir_path = REPO_DIR / "src/common/model_utils"
utils_paths = glob.glob(os.path.join(utils_dir_path, "*.py"))
temp_model_util_dir = Path(temp_path) / "tmp_model_util"
if os.path.exists(temp_model_util_dir):
    shutil.rmtree(temp_model_util_dir)
os.mkdir(temp_model_util_dir)
os.system(f'touch {temp_model_util_dir}/__init__.py')
for p in utils_paths:
    shutil.copy(p, temp_model_util_dir)

# Connect to azure workspace.

Make sure that you have a config.json file with the keys subscription_id, resource_group, and cgm-ml-dev. Either here (not so nice) or in a parent folder (okay but not perfect), or in the root folder of this repo (way to go).

In [6]:
workspace = Workspace.from_config()
workspace

WARNING - Failed : Loading cloud metatdata from the url specified by https://management.azure.com/metadata/endpoints?api-version=2019-05-01 https://management.azure.com/metadata/endpoints?api-version=2019-05-01
WARNING - Failed : Loading cloud metatdata from the url specified by https://management.azure.eaglex.ic.gov/metadata/endpoints?api-version=2019-05-01 https://management.azure.eaglex.ic.gov/metadata/endpoints?api-version=2019-05-01
WARNING - Failed : Loading cloud metatdata from the url specified by https://management.azure.microsoft.scloud/metadata/endpoints?api-version=2019-05-01 https://management.azure.microsoft.scloud/metadata/endpoints?api-version=2019-05-01


Workspace.create(name='cgm-ml-prod-we-azml', subscription_id='9b5bbfae-d5d1-4aae-a2ca-75159c0c887d', resource_group='cgm-ml-prod-we-rg')

# Get the experiment.

- You should always arrange all your runs in an experiment.
- Create at least one experiment per sprint.
- Make sure that the name of the experiment reflects the sprint number.
- On top of that you could also add other tokens to the name. For example network architecture, dataset name, and/or targets.

In [7]:
from azureml.core import Experiment
experiment = Experiment(workspace=workspace, name=experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
q1-depthmap-plaincnn-height-95k-evidential,cgm-ml-prod-we-azml,Link to Azure Machine Learning studio,Link to Documentation


# Find/create a compute target.

Connects to a compute cluster on Azure ML.
If the compute cluster does not exist, it will be created.

Note: Usually computer clusters autoscale. This means that new nodes are created when necessary. And unused VMs will be shut down.

In [8]:
import os
import urllib
import shutil
import azureml
 
from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpu-cluster"

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
        max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

Found existing compute target.


AmlCompute(workspace=Workspace.create(name='cgm-ml-prod-we-azml', subscription_id='9b5bbfae-d5d1-4aae-a2ca-75159c0c887d', resource_group='cgm-ml-prod-we-rg'), name=gpu-cluster, id=/subscriptions/9b5bbfae-d5d1-4aae-a2ca-75159c0c887d/resourceGroups/cgm-ml-prod-we-rg/providers/Microsoft.MachineLearningServices/workspaces/cgm-ml-prod-we-azml/computes/gpu-cluster, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None)

# Get the dataset for training.

Here you specify which dataset to use.

Note: Double check on Azure ML that you are using the right one.

In [9]:
dataset = workspace.datasets[dataset_name]
dataset

{
  "source": [
    "('omdena_datasets', '95k_depthmap_trainingdata/**')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "8c6604b8-d248-410a-9424-09b0bc883369",
    "name": "anon-depthmap-95k",
    "version": 1,
    "description": "A depthmap based dataset containing 95k artifacts, to be used for training.",
    "workspace": "Workspace.create(name='cgm-ml-prod-we-azml', subscription_id='9b5bbfae-d5d1-4aae-a2ca-75159c0c887d', resource_group='cgm-ml-prod-we-rg')"
  }
}

# Push the training source code to Azure.

Creates an estimator (a template for a compute cluster node) and pushes it to the compute cluster.

In [10]:
from azureml.core.runconfig import MpiConfiguration

In [11]:
from src.config import CONFIG, DATASET_MODE_MOUNT, DATASET_MODE_DOWNLOAD, DATASET_MODE_MOUNT

script_params = {f"--{k}": v for k, v in CONFIG.items()}
script_params

{'--DATASET_MODE': 'dataset_mode_download',
 '--DATASET_NAME': 'anon-depthmap-95k',
 '--DATASET_NAME_LOCAL': 'anon-depthmap-95k',
 '--SPLIT_SEED': 0,
 '--IMAGE_TARGET_HEIGHT': 240,
 '--IMAGE_TARGET_WIDTH': 180,
 '--EPOCHS': 1000,
 '--BATCH_SIZE': 256,
 '--SHUFFLE_BUFFER_SIZE': 2560,
 '--NORMALIZATION_VALUE': 7.5,
 '--LEARNING_RATE': 7e-07,
 '--USE_ONE_CYCLE': False,
 '--USE_DROPOUT': True,
 '--USE_WANDB': True,
 '--EDL_COEFF': 0.01,
 '--TARGET_INDEXES': [0]}

In [12]:
if CONFIG.DATASET_MODE == DATASET_MODE_MOUNT:
    print('mount')
    dataset_input = dataset.as_mount()
elif CONFIG.DATASET_MODE == DATASET_MODE_DOWNLOAD:
    print('download')
    dataset_input = dataset.as_download()
else: 
    raise NameError("Unknown DATASET_MODE")

download


In [13]:
curated_env_name = "cgm-env-v15"

ENV_EXISTS = True
if ENV_EXISTS:
    cgm_env = Environment.get(workspace=workspace, name=curated_env_name)
else:
    cgm_env = Environment.from_conda_specification(name=curated_env_name, file_path=REPO_DIR / "environment_train.yml")
    cgm_env.docker.enabled = True
    cgm_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
    cgm_env.register(workspace)  # Please be careful not to overwrite existing environments

In [14]:
# Create the ScriptRunConfig
script_run_config = ScriptRunConfig(source_directory=temp_path,
                                    compute_target=compute_target,
                                    script='train.py',
                                    arguments=[dataset_input]+[str(item) for sublist in script_params.items() for item in sublist],
                                    environment=cgm_env,
)

# Set compute target.
script_run_config.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(config=script_run_config, tags=tags)

# Show run.
run

Experiment,Id,Type,Status,Details Page,Docs Page
q1-depthmap-plaincnn-height-95k-evidential,q1-depthmap-plaincnn-height-95k-evidential_1613407962_cfe86fe2,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
# run.wait_for_completion(show_output=True)

# Delete temp folder.

After all code has been pushed to Azure ML, the temp folder will be removed.

In [16]:
shutil.rmtree(temp_path)